In [2]:
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import numpy as np
import math as m
your_mesh = mesh.Mesh.from_file('cylinder_org.stl', calculate_normals=True)
P0=your_mesh.v0.tolist()
P1=your_mesh.v1.tolist()
P2=your_mesh.v2.tolist()
vert=your_mesh.vectors.tolist()
are=your_mesh.areas.tolist()
area=np.array(are)
cen=np.zeros([len(area),3])
for i in range(len(area)):
    for j in range(3):
        cen[i][j]=(P0[i][j]+P1[i][j]+P2[i][j])/3

        
#your_mesh.get_mass_properties()
cen
usd=your_mesh.get_unit_normals()
un=np.array(usd)

import pandas as pd
d=120
D=15
theta = np.array([0])#summation j
f = np.array([1/12]) #summation i
a= np.array([[1.25]]) # summation i,j (i is differnt frequency),(j is diffetent sets of theta)
TP=1/f
L=np.zeros([len(TP),1])


#Function for calculation of WAVELENGTH
def wavlen(T,DE):
    L0=(1.56)*(T**2)
    lo=L0
    for i in range(10000):
        lne=((9.81*(T**2))/(2*m.pi))*m.tanh((2*m.pi*DE)/lo)
        err=abs(lne-lo)
        if (err<1e-6):
            l=lne
            break
        else:
            lo=lne 
            
            
    return(l)


for i in range(len(TP)):
    L[i]=wavlen(TP[i],d)
#L = np.array(Lt)
k = 2*m.pi/L
w = 2*m.pi*f


gama = 10055.55 
t = np.arange(0,12.1,0.1)
p=np.zeros([len(area),1])
fr=np.zeros([3,1])
tr=np.zeros([3,1])
FORCE=np.zeros([len(t),3])
TORQUE=np.zeros([len(t),3])
Frc=np.zeros([1,3])
Trx=np.zeros([1,3])
com=your_mesh.get_mass_properties()[1]
rco=cen-com


for time in range(len(t)):
    fr=[0,0,0]
    tr=[0,0,0]
    for run in range(len(cen)):
        stat_press = gama*-1*cen[run][2]
        dynamic_pressure =0
        if(cen[run][2]>0):
            dynamic_pressure =0
            stat_press =0
        else:
            for i in range(len(f)):
                for j in range(len(theta)):
                    phase = k[i]*cen[run][0]*np.cos(theta[j]) + k[i]*cen[run][1]*np.sin(theta[j]) - w*t[time]
                    temp = np.cos(phase)
                    Kp = np.cosh((d + (cen[run][2]))*k[i])/np.cosh(k[i]*d)
                    dynamic_pressure += temp*gama*a[i][j]*Kp
        
        p[run]=(dynamic_pressure + stat_press)
        fr[0]+=p[run]*area[run]*un[run][0]
        fr[1]+=p[run]*area[run]*un[run][1]
        fr[2]+=p[run]*area[run]*un[run][2]
        Frc[0][0]=p[run]*area[run]*un[run][0]
        Frc[0][1]=p[run]*area[run]*un[run][1]
        Frc[0][2]=p[run]*area[run]*un[run][2]
        Trx=np.cross(rco[run],Frc[0])
        tr[0]+=Trx[0]
        tr[1]+=Trx[1]
        tr[2]+=Trx[2]
    FORCE[time][0]=fr[0]
    FORCE[time][1]=fr[1]
    FORCE[time][2]=fr[2]
    TORQUE[time][0]=tr[0]
    TORQUE[time][1]=tr[1]
    TORQUE[time][2]=tr[2]
        
FRCE = pd.DataFrame(FORCE,columns = ['FX','FY','FZ'])
TRQ = pd.DataFrame(TORQUE,columns = ['MX','MY','MZ'])
TIM=pd.DataFrame(data=t,columns =['TIME'])
df = pd.concat([TIM,FRCE,TRQ],axis = 1)
#da=abs(df)

    
    

            


 


    

#df.to_excel('force_record.xlsx',index=False)
    


In [3]:
df

,TIME,FX,FY,FZ,MX,MY,MZ
0,0.0,-0.247654,0.000002,-1.115271e+06,0.001308,-84.161027,-62998.145283
1,0.1,1668.238856,-14.750594,-1.115192e+06,8.577534,-6797.871521,-62970.570218
2,0.2,3332.153021,-29.460760,-1.114954e+06,17.130252,-13493.105425,-62887.930333
3,0.3,4986.934168,-44.090176,-1.114558e+06,25.636022,-20151.511569,-62750.452140
4,0.4,6628.046658,-58.598744,-1.114006e+06,34.071528,-26754.839725,-62558.512455
...,...,...,...,...,...,...,...
116,11.6,-6628.528234,58.598747,-1.114006e+06,-34.068914,26587.709989,-62558.551105
117,11.7,-4987.421739,44.090179,-1.114558e+06,-25.633407,19983.861269,-62750.481221
118,11.8,-3332.644886,29.460764,-1.114954e+06,-17.127637,13325.082270,-62887.949765
119,11.9,-1668.733302,14.750598,-1.115192e+06,-8.574918,6629.624243,-62970.579947


In [3]:
com[0]

0.0003297545096852305

In [8]:
your_mesh.get_mass_properties()[2][2]
                            

array([-6.81610301e-06,  2.81110542e-06,  1.95591208e+02])

In [10]:
your_mesh.get_mass_properties()[2][2][2]

195.59120791587367

In [9]:
your_mesh.get_mass_properties()[2]

array([[ 9.23341331e+03,  2.00744315e-07, -6.81610301e-06],
       [ 2.00744315e-07,  9.23339201e+03,  2.81110542e-06],
       [-6.81610301e-06,  2.81110542e-06,  1.95591208e+02]])

In [18]:
df.nlargest(10,columns='FX')

,TIME,FX,FY,FZ,MX,MY,MZ
30,3.0,31880.368305,-281.844398,-1.057363e+06,163.869956,-128338.626567,-42880.650220
31,3.1,31836.693902,-281.458140,-1.054332e+06,163.645376,-128161.393106,-41827.786871
29,2.9,31836.661016,-281.458140,-1.060394e+06,163.645383,-128164.248677,-43933.513825
32,3.2,31705.757517,-280.300425,-1.051310e+06,162.972259,-127633.034079,-40777.809602
28,2.8,31705.691835,-280.300425,-1.063416e+06,162.972273,-127638.737394,-44983.491857
33,3.3,31487.918037,-278.374426,-1.048304e+06,161.852450,-126754.997681,-39733.596327
27,2.7,31487.819739,-278.374426,-1.066422e+06,161.852470,-126763.533107,-46027.706402
34,3.4,31183.772544,-275.685422,-1.045323e+06,160.289018,-125529.690548,-38698.009163
26,2.6,31183.641900,-275.685422,-1.069403e+06,160.289045,-125541.034691,-47063.295340
35,3.5,30794.154681,-272.240784,-1.042375e+06,158.286249,-123960.471163,-37673.886583


In [19]:
df.nsmallest(10,columns='FX')

,TIME,FX,FY,FZ,MX,MY,MZ
90,9.0,-31880.235248,281.844398,-1.057364e+06,-163.867466,128224.866827,-42880.836120
91,9.1,-31836.560845,281.458140,-1.060394e+06,-163.642887,128047.633366,-43933.699470
89,8.9,-31836.527959,281.458140,-1.054333e+06,-163.642893,128050.488937,-41827.972516
92,9.2,-31705.624460,280.300425,-1.063417e+06,-162.969770,127519.274339,-44983.676739
88,8.8,-31705.558778,280.300425,-1.051310e+06,-162.969783,127524.977654,-40777.994483
93,9.3,-31487.784980,278.374426,-1.066422e+06,-161.849961,126641.237941,-46027.890013
87,8.7,-31487.686682,278.374426,-1.048305e+06,-161.849980,126649.773367,-39733.779938
94,9.4,-31183.639487,275.685422,-1.069403e+06,-160.286529,125415.930808,-47063.477177
86,8.6,-31183.508842,275.685422,-1.045324e+06,-160.286555,125427.274951,-38698.191001
95,9.5,-30794.021624,272.240784,-1.072351e+06,-158.283759,123846.711423,-48087.599758
